In [8]:
%load_ext autoreload

# Introduction to the `BrainGlobeAtlas` class

## 0. Creating a `BrainGlobeAtlas` object and list availabe options

To instantiate a `BrainGlobeAtlas` object, we need to instantiate it with the atlas name. The first time we use it, a version of this atlas files will be downloaded from the [remote GIN repository](http://gin.g-node.org/brainglobe/atlases) and stored on your local machine (by default, in .../Users/username/.brainglobe):

In [21]:
%autoreload
from bg_atlasapi import BrainGlobeAtlas

bg_atlas = BrainGlobeAtlas("example_mouse_100um", check_latest=False)

example mouse atlas (res. 100um)
From: http://www.brain-map.org.com (Wang et al 2020, https://doi.org/10.1016/j.cell.2020.04.007 )


To know what atlases are available through BrainGlobe, we can use the `show_atlases` function (we need to be online):

In [7]:
from bg_atlasapi import show_atlases

In [8]:
show_atlases()

## 1. Using a `BrainGlobe` atlas

A BrainGlobe atlas is a convenient API for interacting with an anatomical atlas. BrainGlobe atlases contain:
 * Metadata
 * Reference anatomical stack
 * Region annotation stack
 * Hemisphere annotation stack
 * Description of the region hierarchy
 * Meshes for the regions

### 1.0 Metadata

In [22]:
bg_atlas.metadata

{'name': 'example_mouse',
 'citation': 'Wang et al 2020, https://doi.org/10.1016/j.cell.2020.04.007',
 'atlas_link': 'http://www.brain-map.org.com',
 'species': 'Mus musculus',
 'symmetric': True,
 'resolution': [100.0, 100.0, 100.0],
 'orientation': 'asl',
 'version': '0.3',
 'shape': [132, 80, 114],
 'trasform_to_bg': [[1.0, 0.0, 0.0, 0.0],
  [0.0, 1.0, 0.0, 0.0],
  [0.0, 0.0, 1.0, 0.0],
  [0.0, 0.0, 0.0, 1.0]]}

### 1.1 Anatomical stack